In [1]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell

# 2. run this cell
%cd ..

from domblar.domblar import Domblar
d = Domblar(
    synth_count=4,
    context='dexed',
)

/Users/gexahedron/devel/domblar


In [3]:
# comma pump examples from Paul Erlich's "A Middle Path" paper

import math
import time
from copy import deepcopy
import numpy as np

def set_synths():
    for i in range(4):
        d.set_synth(i, 'harpic3')
        # d.set_synth(i, 'brass3')
set_synths()


WESTERN_EDO = 12
OCTAVE_IN_CENTS = WESTERN_EDO * 100
A4_MIDI = 69
A4_FREQ = 440.0
C4_FREQ = A4_FREQ * (2 ** (-9 / WESTERN_EDO))


# this is from music_math.py
# def calc_hz(pitch, ref_hz):
#     if pitch is None:
#         return None
#     return ref_hz * (2 ** (pitch / OCTAVE_IN_CENTS))

# FIXME: rename
def calc_hz(ji_shift_orig, base_hz):
    ji_shift = deepcopy(ji_shift_orig)
    cur_hz = base_hz
    low_hz = 20
    high_hz = 10000
    while any(ji_shift):
        changed = False
        for i, p in enumerate(primes):
            if ji_shift[i] != 0:
                if cur_hz < low_hz and ji_shift[i] < 0:
                    continue
                if cur_hz > high_hz and ji_shift[i] > 0:
                    continue
                if ji_shift[i] > 0:
                    cur_hz *= float(p)
                    ji_shift[i] -= 1
                else:
                    cur_hz /= float(p)
                    ji_shift[i] += 1
                changed = True
        if not changed:
            print('not changed!', cur_hz, ji_shift)
        assert(changed)
    return cur_hz

def ratio_to_cents(m, n):
    return OCTAVE_IN_CENTS * math.log2(float(m) / n)


def parse(scale):
    ratios = scale.split(', ')
    ratios = [r.split('/') for r in ratios]
    ratios = [(int(r[0]), int(r[1])) for r in ratios]
    return ratios

# scale here is actually a collection of ratios,
# which are used below in the comma pump
scale = '1/1, 6/5, 3/2, 5/4, 2/3, 3/4, 3/5, 7/8, 7/4, 5/8, 8/7, 8/5, 4/3'
ratios = parse(scale)
ratios.sort(key=lambda r: ratio_to_cents(r[0], r[1]))


primes = [2, 3, 5, 7, 11, 13]
spectrums = dict()
for r in ratios:
    spectrum = [0] * (len(primes))
    nr = [r[0], r[1]]
    for i, p in enumerate(primes):
        while nr[0] % p == 0:
            spectrum[i] += 1
            nr[0] //= p
        while nr[1] % p == 0:
            spectrum[i] -= 1
            nr[1] //= p
    spectrums[r] = np.array(spectrum)


# format
# (a, b)
# a - lists any note which goes further and where it goes
# b - chord, which always includes fraction 1/1, the new reference tonic

# chords = [
#     ([0, 0], [(7, 8), (1, 1), (5, 4), (3, 2)]),
#     ([3, 3], [(1, 1), (5, 4), (3, 2), (7, 4)]),
#     ([2, 3], [(5, 8), (3, 4), (7, 8), (1, 1)]),
#     ([1, 1], [(5, 8), (3, 4), (7, 8), (1, 1)]),
#     ([2, 2], [(1, 1), (8, 7), (4, 3), (8, 5)]),
#     ([1, 0], [(7, 8), (1, 1), (5, 4), (3, 2)]),
#     ([2, 2], [(1, 1), (8, 7), (4, 3), (8, 5)]),
# ]
# first_chord_is_longer = True

# TODO:
# this chord progression could be analyzed in 12edo from Neo-Riemannian point of view
chords = [
    ([1, 0], [(1, 1), (6, 5), (3, 2)]),
    ([0, 0], [(1, 1), (5, 4), (3, 2)]), # пояснение ко второй ноте 5/4: 3/2 = 5/4 * 6/5
    ([1, 1], [(2, 3), (3, 4), (1, 1)]),
    ([0, 0], [(3, 5), (3, 4), (1, 1)]),
    ([0, 0], [(1, 1), (5, 4), (3, 2)]),
    ([1, 1], [(2, 3), (3, 4), (1, 1)]),
    ([0, 0], [(3, 5), (3, 4), (1, 1)]),
    ([0, 0], [(1, 1), (5, 4), (3, 2)]),
]
first_chord_is_longer = False

cur_chord = [None] * len(chords[0][1])
prev_chord = deepcopy(cur_chord)

initial_tonic_in_hz = A4_FREQ
cur_tonic_in_ji_shift = np.array([0, 0, 0, 0, 0, 0])

# TODO:
# mos = miracle21

send_note_dur = 0.25
amp = 0.5

first = True
reps = 8

for iteration in range(reps):
    for i, chord_info in enumerate(chords):
        note_map, chord = chord_info
        if not first:
            i1 = 0
            for j, n in enumerate(cur_chord):
                if n is not None:
                    i1 = j
                    break
            cur_tonic_in_ji_shift = cur_chord[i1] - spectrums[chord[i1]]
        first = False

        cur_chord = np.array([cur_tonic_in_ji_shift] * len(cur_chord))
        for j in range(len(chord)):
            cur_chord[j] += spectrums[chord[j]]

        ji_hzs = []
        mos_hzs = []
        for n in cur_chord:
            ji_hz = calc_hz(n, initial_tonic_in_hz)
            ji_hzs.append(ji_hz)
            # TODO:
            # tempered_note_in_cents = mos.calc_note_in_cents(n)
            # mos_hz = calc_hz_from_cents(tempered_note_in_cents, initial_tonic_in_hz)
            # mos_hzs.append(mos_hz)
        print(ji_hzs)
        dur = send_note_dur
        if i == 0 and first_chord_is_longer:
            dur *= 2

        # TODO:
        # for synth_idx, freq in enumerate(mos_hzs):
        for synth_idx, freq in enumerate(ji_hzs):
            timetag = time.time()
            d.client.send_note(
                synth_idx,
                freq=freq, dur=dur, amp=amp,
                timetag=timetag, channel=0)
        time.sleep(dur)

        # print(i, 'mos hz', mos_hzs)

        prev_chord = deepcopy(cur_chord)
        cur_chord = [None] * len(cur_chord)
        cur_chord[note_map[1]] = prev_chord[note_map[0]]

        if i == 0 and iteration == reps - 1:
            break


[440.0, 528.0, 660.0]
[528.0, 660.0, 792.0]
[528.0, 594.0, 792.0]
[475.20000000000005, 594.0, 792.0]
[475.20000000000005, 594.0, 712.8000000000001]
[475.20000000000005, 534.6, 712.8000000000001]
[427.68, 534.6, 712.8000000000001]
[427.68, 534.6, 641.52]
[427.68, 513.216, 641.52]
[513.216, 641.52, 769.8240000000001]
[513.216, 577.368, 769.8240000000001]
[461.8944, 577.368, 769.8240000000001]
[461.8944, 577.368, 692.8416]
[461.8944, 519.6312, 692.8416]
[415.7049600000001, 519.6312, 692.8416]
[415.7049600000001, 519.6312, 623.5574400000002]
[415.7049600000001, 498.84595200000007, 623.5574400000002]
[498.84595200000007, 623.5574400000002, 748.2689280000001]
[498.84595200000007, 561.2016960000001, 748.2689280000001]
[448.9613568000001, 561.2016960000001, 748.2689280000001]
[448.9613568000001, 561.2016960000001, 673.4420352000002]
[448.9613568000001, 505.08152640000014, 673.4420352000002]
[404.06522112000005, 505.08152640000014, 673.4420352000002]
[404.06522112000005, 505.08152640000014, 606

In [4]:
# cleanup
d.stop_server()
